In [1]:
import os
import glob
import json
import time

import cv2
import numpy as np
try:
    import labelbox 
except ImportError:
    print("Warning labelbox not installed.")
    
from koger_detection.utils.video import extract_crops
from koger_detection.labelbox.annotations import

# def extract_crops(video_names, video_folder_path, save_folder, num_extract, 
#                   crop_size, min_frame=0, max_frame=None, save_triplet=False, 
#                   triplet_spacing=30):
#     """Extract random crops from video clips.
    
#     Can extract frames a set time before and after each crop to
#     make annotation easier so the annotator can use local movement.
    
#     video_names: list of names of video clips
#     video_folder_path: full path to folder where videos are stored
#     save_folder: full path to folder where crops should be saved
#     num_extract: how many random focal frames to extract from each
#         video clip
#     crop_size: length of square crop in pixels
#     min_frame: min frame number in video frames should be chosen from 
#     max_frame: max frame number in video frames should be chosen from
#     save_triplet: if True, save frames a fixed number of frames before
#         and after focal frame to aid in annoation effort
#     triplet_spacing: how many frames before and after focal frame triplet
#         frames should be. (Only relevant if triplet spacing is True)
    
#     """
#     for video_name in video_names:
#         video_file = os.path.join(video_folder_path, video_name)
#         video_name = os.path.splitext(video_name)[0] # remove extension

#         cap = cv2.VideoCapture(video_file)
#         width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
#         height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
#         if max_frame is None:
#             max_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) + 1
#         if save_triplet:
#             # Only choose frames for annotation that have space before and after
#             # for all frames in triplet
#             max_frame -= triplet_spacing - 1
#             min_frame += triplet_spacing
#         # Randomly choose the specified number of frames to extract from the given range
#         frame_nums_to_save = np.random.randint(min_frame, max_frame, num_extract)
#         for frame_num in frame_nums_to_save:
#             frame_file = os.path.join(save_folder, f"{video_name}_frame_{frame_num}")
#             if crop_size:
#                 # If gaussian is True, then random crops sampled from gaussian 
#                 # centered at center of frame with 1 std equal to half height/width 
#                 # of the frame
#                 top_left = random_top_left([height, width], crop_size, gaussian=False)
#                 if top_left is False:
#                     print(f"skipping {frame_file}")
#                     print(f"No valid crop found. Check frame size and crop size.")
#                     print(f"Frame size h:{height}, w:{width}, crop_size: {crop_size}.")
#                     continue
#                 # Add where crop comes from to file name so we can find it in the 
#                 # original image later if we want to
#                 frame_file += f"_top_{top_left[0]}_left_{top_left[1]}"
#             # Naming convention here is to append an 'f' if the focal frame that will
#             # be annotated and a 'a' or 'b' if the first or last frame in a triplet
#             save_frame(cap, frame_num, frame_file+"_f.jpg", crop_size, top_left)
#             if save_triplet:
#                 next_frame_num = frame_num + triplet_spacing
#                 frame_file = os.path.join(save_folder, f"{video_name}_frame_{frame_num}")
#                 if crop_size:
#                     frame_file += f"_top_{top_left[0]}_left_{top_left[1]}"
#                 save_frame(cap, next_frame_num, frame_file+"_a.jpg", 
#                            crop_size, top_left
#                           )
#                 prev_frame_num = frame_num - triplet_spacing 
#                 frame_file = os.path.join(save_folder, f"{video_name}_frame_{frame_num}")
#                 if crop_size:
#                     frame_file += f"_top_{top_left[0]}_left_{top_left[1]}"
#                 save_frame(cap, prev_frame_num, frame_file+"_b.jpg",
#                            crop_size, top_left
#                           )
#         cap.release()
    

# def random_top_left(im_shape, crop_size, gaussian=False):
#     """ Get a random top left coordinate for a crop of size (crop_size * crop_size).
    
#     Args:
#         im_shape: (h, w, ...)
#         crop_size: size of ultimate crop
#         gaussian: If True, then pull coordinates from gaussian with mean
#             at the center of possible range of top left values and 1 standard 
#             deviation to the min and max top left values
    
#     Returns [top, left] 
#         or False if no valid top_left value based on image and crop size
#     """
    
#     height, width = im_shape[:2]
#     if gaussian:
#         mean_top = (width-crop_size) / 2
#         mean_left = (width-crop_size) / 2
#         top = -1
#         left = -1
#         while ((top >= (height-crop_size)) or (top < 0)):
#             top = int(np.random.normal(mean_top, mean_top))
#         while ((left >= (width-crop_size)) or (left < 0)):
#             left = int(np.random.normal(mean_left, mean_left))
#     else:
#         if height - crop_size < 0:
#             return False
#         top = np.random.randint(0, height-crop_size)

#         if width - crop_size < 0:
#             return False
#         left = np.random.randint(0, width-crop_size)

#     top_left = [top, left]
#     return top_left


# def save_frame(cap, frame_num, outfile, crop_size=None, top_left=None):
#     """ Save frame from cv2 VideoCapture
    
#     Args: 
#         cap: cv2.VideoCapture object
#         frame_num: the frame number to save
#         outfile: where to save frame
#         crop_size: pixels, size of crop (square). If None, no crop.
#         top_left: (i, j) coordinate of top left corner of crop (if not None)
#             if None and crop_size is not None, then choose random values
            
#     Return crop_top_left
        
#     """
#     cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
#     ret, frame = cap.read()
#     if frame is not None:
#         if crop_size:
#             if not top_left:
#                 raise ValueError(f"If cropping, must provide top_left: {top_left}")
#             top, left = top_left
#             frame = frame[top:top+crop_size, left:left+crop_size]

#         os.makedirs(os.path.dirname(outfile), exist_ok=True)
#         cv2.imwrite(outfile, frame)
#     else:
#         print(f"Frame doesn't exist {outfile}.")
        

# def create_storage_dataset(labelbox_client, image_files, dataset_name):
#     """First create dataset with images including overlay images
#     so that they are given an online adress by labelbox. Nessisary
#     step to eventually have overlay dataset for focal frames.
    
#     Args:
#         labelbox_client: labelbox Client object
#         image_files: list of files to upload to labelbox dataset
#         dataset_name: name of the final overlay dataset to be annotated
#             ('-storage' will be appended)
#     """
    
#     # This dataset is just for uploading all images to the cloud
#     # In a second dataset some of these images will become overlay images
#     # So the user can use animal movement to help detect crypric individuals
#     storage_dataset = labelbox_client.create_dataset(name=f"{dataset_name}-storage")

#     # Create data payload
#     # External ID is recommended to identify your data
#     my_data_rows = []
#     for image_file in image_files:
#         my_data_rows.append({"row_data": image_file,
#                              "external_id": os.path.basename(image_file)
#                             }
#                            )
#     # Bulk add data rows to the dataset
#     task = storage_dataset.create_data_rows(my_data_rows)
#     task.wait_till_done()
#     while task.status != "COMPLETE":
#         print(f"Storage dataset upload status: {task.status}")
#         time.sleep(3)
#     return task.status == "COMPLETE"
        
        
# def add_crops_to_labelbox(labelbox_client, focal_images, dataset_name):
#     """ Create labelbox dataset with overlay images around focal image.
    
#     Args: 
#         labelbox_client: labelbox Client object
#         focal_images: list of focal image full paths
#             (These are the files that are going to actually be 
#             annotated (marked by _f))
#         dataset_name: name of the dataset 
            
#         """
    
#     focal_names = [os.path.basename(f) for f in focal_images]
    
#     dataset = labelbox_client.create_dataset(name=dataset_name)
    
#     storage_ds = labelbox_client.get_datasets(
#         where=labelbox.Dataset.name == f"{dataset_name}-storage").get_one()

#     my_data_rows = []

#     for focal_name in focal_names:
#         prev_name = focal_name.split("_f.jpg")[0] + "_a.jpg"
#         next_name = focal_name.split("_f.jpg")[0] + "_b.jpg"
#         data_row = storage_ds.data_row_for_external_id(focal_name)
#         prev_row = storage_ds.data_row_for_external_id(prev_name)
#         next_row = storage_ds.data_row_for_external_id(next_name)

#         my_data_rows.append({"row_data": data_row.row_data,
#                              "external_id": data_row.external_id,
#                              "attachments": [
#                                  {
#                                      "type": "IMAGE_OVERLAY",
#                                      "value": prev_row.row_data
#                                  },
#                                  {
#                                      "type": "IMAGE_OVERLAY",
#                                      "value": next_row.row_data
#                                  }
#                              ]
#                             }
#                            )

#     # Bulk add data rows to the dataset
#     task = dataset.create_data_rows(my_data_rows)

#     task.wait_till_done()
#     while task.status != "COMPLETE":
#         print(f"Storage dataset upload status: {task.status}")
#         time.sleep(3)
#     return f"successful upload: {task.status == 'COMPLETE'}"
    

In [3]:
import json

# This file records which raw video files should be used for 
# training, validation, and testing
# Format of .json is:
# {"train": ["file1.mp4", "file2.mp4"], 
#  "val": ["file3.mp4", "file4.mp4"],
#  "test": ["file5.mp4", "file6.mp4"]
# }

# can instead just manually create a list of video names
# and pass that to the variable 'video_names' below
json_file = "/home/koger/Dropbox/UWWRF/detection/high-altitude/high-altitude-overhead/detection/model-training/video_train_val_test_split.json"
with open(json_file) as f:
    video_splits = json.load(f)

In [4]:
# Path to main folder within which images will be saved
annotations_folder = "/home/koger/Dropbox/UWWRF/detection/high-altitude/annotations/extracted_frames/crops"
os.makedirs(annotations_folder, exist_ok=True)
# Path to folder where videos are saved
video_folder_path = "/media/koger/HDD2022_3/bristolbay"

# Which type of videos to extract from based on naming in our saved .json file
# this could be "train", "validate", or "test"
project_name = "salmon-high-alt"
data_type = "train-all"
annotation_round = 1

dataset_name = f"{project_name}-{data_type}_{annotation_round}"

# list of video names to use
video_names = video_splits[data_type]

save_folder = os.path.join(annotations_folder, dataset_name)
# setting min and max frames can be used if part of video will go in the training set
# and part of the video will go in the validation set
min_frame = 0 # begining of range frames could be extracted from
max_frame = None # end of range (exclusive) frames could be extracted from (
                 # set to "None" if range should extend through last frame)
num_extract = 7 # number of frames to extract and save
save_triplet = True # If True, for each frame should save a frame a little before 
                    # and a little after focal frame (Can be helpful when 
                    # movement helps find cryptic individuals)
triplet_spacing = 20 # How many frames in future and past to space outer 
                     # triplet frames around focal frame (ignored if 
                    # save_triplet is False)
crop_size = 1024 # Either none for whole image or size in pixels (square crops)

In [5]:
extract_crops(video_names, video_folder_path, save_folder, 
              num_extract, crop_size, min_frame=min_frame, max_frame=max_frame,
              save_triplet=save_triplet, triplet_spacing=triplet_spacing)

Frame doesn't exist /home/koger/Dropbox/UWWRF/detection/high-altitude/annotations/extracted_frames/crops/salmon-high-alt-train-all_1/confluence_double_2022_08_10_berdahl_02_3_DJI_0781_frame_4945_top_841_left_1976_f.jpg.
Frame doesn't exist /home/koger/Dropbox/UWWRF/detection/high-altitude/annotations/extracted_frames/crops/salmon-high-alt-train-all_1/confluence_double_2022_08_10_berdahl_02_3_DJI_0781_frame_4945_top_841_left_1976_a.jpg.
Frame doesn't exist /home/koger/Dropbox/UWWRF/detection/high-altitude/annotations/extracted_frames/crops/salmon-high-alt-train-all_1/confluence_double_2022_08_10_berdahl_02_3_DJI_0781_frame_4945_top_841_left_1976_b.jpg.
Frame doesn't exist /home/koger/Dropbox/UWWRF/detection/high-altitude/annotations/extracted_frames/crops/salmon-high-alt-train-all_1/confluence_double_2022_08_10_berdahl_02_3_DJI_0781_frame_3475_top_535_left_152_f.jpg.
Frame doesn't exist /home/koger/Dropbox/UWWRF/detection/high-altitude/annotations/extracted_frames/crops/salmon-high-alt-

### Upload frames to labelbox

In [6]:
import glob
import os

from dotenv import load_dotenv
from labelbox import Client

In [7]:
load_dotenv()

client = Client(api_key=os.environ.get('LABELBOX_API_KEY'))

In [8]:
# save_folder = "/home/koger/Dropbox/UWWRF/detection/high-altitude/annotations/extracted_frames/crops/salmon-high-alt-train-3_1"
crop_files = sorted(glob.glob(os.path.join(save_folder, "*.jpg")))

In [9]:
# Uploads focal frames and 
create_storage_dataset(client, crop_files, dataset_name)

True

In [10]:
focal_images = sorted(glob.glob(os.path.join(save_folder, "*_f.jpg")))

In [11]:
add_crops_to_labelbox(client, focal_images, dataset_name)

'successful upload: True'